# Embeddings: Representaciones Vectoriales de Texto

## Introducción

### ¿Qué son los Embeddings?

Los **embeddings** son una tecnología fundamental en el procesamiento de lenguaje natural (NLP) que permite convertir texto en representaciones matemáticas que las máquinas pueden entender y procesar eficientemente.

**Definición Simple**: Un embedding es una representación vectorial (lista de números) de un texto que captura su significado semántico.

### ¿Por qué son Importantes?

1. **Comprensión Semántica**: Capturan el significado, no solo las palabras
2. **Búsqueda Inteligente**: Encuentran contenido relacionado por significado
3. **Agrupación**: Organizan información similar automáticamente
4. **RAG (Retrieval-Augmented Generation)**: Base para sistemas de IA conversacional

### Objetivos de este Tutorial

En este notebook aprenderás a:

1. 🎯 **Comprender** qué son los embeddings y cómo funcionan
2. 🔧 **Generar** embeddings usando modelos de OpenAI (2025)
3. 📊 **Visualizar** cómo se agrupan los textos por significado
4. 🔍 **Implementar** búsqueda semántica básica
5. 💰 **Comparar** modelos y costos

### Casos de Uso Reales

- **Chatbots**: Entender preguntas de usuarios
- **Recomendaciones**: Sugerir contenido similar
- **Traducción**: Encontrar equivalencias semánticas
- **Análisis de Sentimientos**: Clasificar opiniones

In [ ]:
#!pip install plotly openai langchain-openai scikit-learn pandas umap-learn

## Paso 2: Importación de Bibliotecas

### Importación de Bibliotecas

Vamos a importar las bibliotecas necesarias para este proyecto.

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from openai import OpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from sklearn.decomposition import PCA
import pandas as pd
import umap

In [ ]:
import os
from dotenv import load_dotenv

# Carga las variables de entorno desde un archivo .env si existe
load_dotenv()

## Paso 3: Configuración de OpenAI

### Modelos de Embeddings de OpenAI (2025)

OpenAI ofrece dos modelos principales de embeddings:

- **text-embedding-3-small**: Modelo más eficiente y económico
  - Dimensiones: hasta 1536
  - Precio: $0.00002 por 1K tokens
  - Ideal para aplicaciones generales

- **text-embedding-3-large**: Modelo de mayor rendimiento
  - Dimensiones: hasta 3072
  - Precio: $0.00013 por 1K tokens
  - Mejor precisión para casos complejos

**Características clave:**
- Soporte para dimensiones personalizables
- Mejor rendimiento que modelos anteriores
- Técnica de Matryoshka Representation Learning (MRL)

In [ ]:
# Configurar cliente de OpenAI
client = OpenAI()

# Configurar clientes de embeddings con LangChain
embeddings_small = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=256  # Dimensiones reducidas para visualización
)

embeddings_large = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=3072  # Dimensiones reducidas para visualización
)

In [ ]:
test = embeddings_small.embed_documents(["Hola mundo", "¿Cómo estás?", "¡Buen día!"])

len(test[0])

In [ ]:
# Después de la sección de configuración de OpenAI, añadir:

def get_embedding_models_info():
    """
    Información actualizada de modelos de embeddings (2025)
    """
    models_info = {
        "text-embedding-3-small": {
            "max_dimensions": 1536,
            "price_per_1k_tokens": 0.00002,
            "performance": "Bueno",
            "use_case": "Aplicaciones generales, búsqueda rápida"
        },
        "text-embedding-3-large": {
            "max_dimensions": 3072,
            "price_per_1k_tokens": 0.00013,
            "performance": "Excelente",
            "use_case": "Alta precisión, aplicaciones críticas"
        }
    }
    
    # Crear tabla comparativa
    df_models = pd.DataFrame(models_info).T
    return df_models

# Mostrar información
models_df = get_embedding_models_info()
print("📊 Comparación de Modelos de Embeddings OpenAI (2025)")
print(models_df.to_string())

## Paso 4: ¿Qué son los Embeddings?

### Concepto Fundamental

Los **embeddings** son representaciones vectoriales de texto que capturan significado semántico:

- **Vector numérico**: Cada palabra/frase se convierte en una lista de números reales
- **Espacio semántico**: Textos similares tienen vectores cercanos
- **Dimensiones**: Cada posición del vector captura diferentes aspectos del significado
- **Similitud**: Se mide usando distancias como coseno o euclidiana

### ¿Cómo funcionan?

1. **Entrada**: Texto en lenguaje natural
2. **Procesamiento**: Modelo de IA analiza contexto y significado
3. **Salida**: Vector de números (ej: [0.1, -0.3, 0.8, ...])
4. **Uso**: Búsqueda, comparación, agrupación semántica

In [ ]:
def generate_openai_embeddings(texts, model="text-embedding-3-small", dimensions=512):
    """
    Genera embeddings usando OpenAI y LangChain
    
    Args:
        texts: Lista de textos para convertir en embeddings
        model: Modelo de OpenAI a usar
        dimensions: Número de dimensiones del embedding
    
    Returns:
        np.array: Array de embeddings
    """
    embeddings_client = OpenAIEmbeddings(
        model=model,
        dimensions=dimensions
    )
    
    embeddings = embeddings_client.embed_documents(texts)
    return np.array(embeddings)

def calculate_cosine_similarity(vec1, vec2):
    """Calcula similitud coseno entre dos vectores"""
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

In [ ]:
vector_01 = [1,0,1,1]
vector_02 = [0,0,1,0]
vector_03 = [1,0,1,0]

calculate_cosine_similarity(vector_01, vector_03)

## Paso 5: Textos de Ejemplo para Aprendizaje

### Textos Diversos para Explorar Relaciones Semánticas

Usaremos textos de diferentes dominios para entender cómo los embeddings capturan significado:

In [ ]:
# Textos organizados por categorías semánticas
texts = [
    # Animales domésticos
    "El gato está durmiendo en el sofá de la sala",
    "Mi perro está ladrando en el jardín trasero",
    "El cachorro juega con una pelota en el parque",
    
    # Tecnología
    "La computadora está procesando datos complejos",
    "El smartphone tiene una batería de larga duración",
    "La inteligencia artificial está revolucionando la medicina",
    
    # Naturaleza
    "El cielo está azul y claro esta mañana",
    "Las montañas están cubiertas de nieve blanca",
    "El océano tiene olas muy grandes hoy",
    
    # Educación/Aprendizaje
    "Estamos aprendiendo sobre embeddings de texto",
    "Los estudiantes están estudiando matemáticas avanzadas",
    "La profesora explica conceptos de machine learning",

    # Temporalidad
    "Ayer fuimos al cine con mis amigos",
    "La semana pasada terminé un proyecto importante",
    "Ayer llovió durante toda la tarde",
    "La semana pasada hubo una conferencia sobre IA"
]

print(f"Total de textos: {len(texts)}")
for i, text in enumerate(texts, 1):
    print(f"{i:2d}. {text}")

## Paso 6: Generar Embeddings con OpenAI

### Comparación entre Modelos text-embedding-3-small y text-embedding-3-large

Generaremos embeddings usando ambos modelos para comparar sus representaciones.

In [ ]:
# Generar embeddings con ambos modelos
print("Generando embeddings con text-embedding-3-small...")
embeddings_small = generate_openai_embeddings(texts, model="text-embedding-3-small", dimensions=1536)

print("Generando embeddings con text-embedding-3-large...")
embeddings_large = generate_openai_embeddings(texts, model="text-embedding-3-large", dimensions=3072)

print(f"\nInformación de los embeddings:")
print(f"Modelo small - Shape: {embeddings_small.shape}")
print(f"Modelo large - Shape: {embeddings_large.shape}")

# Ejemplo: mostrar primeras 5 dimensiones del primer texto
print(f"\nPrimeras 5 dimensiones del texto 1 con modelo small:")
print(embeddings_small[0][:5])
print(f"\nPrimeras 5 dimensiones del texto 1 con modelo large:")
print(embeddings_large[0][:5])

In [ ]:
"El gato está durmiendo en el sofá de la sala"

embeddings_large[0][:5]

In [ ]:
from pprint import pprint

print("Texto original:")
print(texts[0])
print("\nPrimeras 10 dimensiones del embedding (large):")
pprint(embeddings_large[0][:10])

## Paso 7: Visualización de Embeddings

### Reducción de Dimensionalidad

Para visualizar embeddings de 512 dimensiones en 2D, usamos técnicas de reducción de dimensionalidad:

- **PCA (Principal Component Analysis)**: Método lineal, rápido, preserva varianza global
- **UMAP (Uniform Manifold Approximation)**: Método no-lineal, preserva estructura local y global

### ¿Por qué es Importante Visualizar?

1. **Comprensión Intuitiva**: Ver cómo se agrupan textos similares
2. **Validación**: Verificar que el modelo captura relaciones semánticas
3. **Debugging**: Identificar problemas en los datos o modelo
4. **Comunicación**: Explicar conceptos a stakeholders

## Paso 8: Comparación Visual - PCA vs UMAP

### Diferencia entre PCA y UMAP

- **PCA (Principal Component Analysis)**:

    - Es un método lineal de reducción de dimensionalidad.
    - Busca las direcciones (componentes principales) que maximizan la varianza global de los datos.
    - Es rápido y fácil de interpretar, pero puede no capturar relaciones complejas o no lineales.
    - Útil para entender la estructura global y la varianza principal de los datos.

- **UMAP (Uniform Manifold Approximation and Projection)**:

    - Es un método no lineal basado en teoría de grafos y topología.
    - Preserva tanto la estructura local como global de los datos en espacios de alta dimensión.
    - Puede capturar agrupaciones y relaciones complejas que PCA no detecta.
    - Es especialmente útil para visualizar agrupaciones semánticas y relaciones no lineales en los embeddings.

In [ ]:
def create_visualization_comparison(embeddings, texts, title_prefix=""):
    """
    Crea visualizaciones comparativas usando PCA y UMAP
    """
    # Crear categorías para colorear
    categories = []
    for text in texts:
        if any(word in text.lower() for word in ['gato', 'perro', 'cachorro']):
            categories.append('Animales')
        elif any(word in text.lower() for word in ['computadora', 'smartphone', 'inteligencia artificial']):
            categories.append('Tecnología')
        elif any(word in text.lower() for word in ['cielo', 'montaña', 'océano']):
            categories.append('Naturaleza')
        elif any(word in text.lower() for word in ['ayer', 'semana pasada', 'cine', 'proyecto', 'conferencia', 'llovió']):
            categories.append('Temporalidad')
        else:
            categories.append('Educación')
    
    # PCA
    pca = PCA(n_components=2, random_state=42)
    embeddings_pca = pca.fit_transform(embeddings)
    
    # UMAP
    umap_reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=5, min_dist=0.1)
    embeddings_umap = umap_reducer.fit_transform(embeddings)
    
    # Crear DataFrames
    df_pca = pd.DataFrame({
        'x': embeddings_pca[:, 0],
        'y': embeddings_pca[:, 1],
        'text': texts,
        'category': categories,
        'method': 'PCA'
    })
    
    df_umap = pd.DataFrame({
        'x': embeddings_umap[:, 0],
        'y': embeddings_umap[:, 1],
        'text': texts,
        'category': categories,
        'method': 'UMAP'
    })
    
    # Visualización PCA
    fig_pca = px.scatter(
        df_pca, x='x', y='y', 
        color='category',
        hover_data=['text'],
        title=f'{title_prefix} - Reducción con PCA',
        labels={'x': f'PC1 ({pca.explained_variance_ratio_[0]:.1%} varianza)', 
                'y': f'PC2 ({pca.explained_variance_ratio_[1]:.1%} varianza)'}
    )
    fig_pca.update_traces(textposition="top center")
    fig_pca.show()
    
    # Visualización UMAP
    fig_umap = px.scatter(
        df_umap, x='x', y='y',
        color='category',
        hover_data=['text'],
        title=f'{title_prefix} - Reducción con UMAP',
        labels={'x': 'UMAP 1', 'y': 'UMAP 2'}
    )
    fig_umap.update_traces(textposition="top center")
    fig_umap.show()
    
    return df_pca, df_umap

# Visualizar embeddings del modelo small
print("=== Visualización: text-embedding-3-small ===")
df_pca_small, df_umap_small = create_visualization_comparison(embeddings_large, texts, "OpenAI Small")

In [ ]:
texts = [
    # Animales domésticos
    "El gato está durmiendo en el sofá de la sala",
    "Mi perro está ladrando en el jardín trasero",
    "El cachorro juega con una pelota en el parque",
    
    # Tecnología
    "La computadora está procesando datos complejos",
    "El smartphone tiene una batería de larga duración",
    "La inteligencia artificial está revolucionando la medicina",
    
    # Naturaleza
    "El cielo está azul y claro esta mañana",
    "Las montañas están cubiertas de nieve blanca",
    "El océano tiene olas muy grandes hoy",
    
    # Educación/Aprendizaje
    "Estamos aprendiendo sobre embeddings de texto",
    "Los estudiantes están estudiando matemáticas avanzadas",
    "La profesora explica conceptos de machine learning",

    # Temporalidad
    "Ayer fuimos al cine con mis amigos",
    "La semana pasada terminé un proyecto importante",
    "Ayer llovió durante toda la tarde",
    "La semana pasada hubo una conferencia sobre IA"
]

## Paso 9: Comparación entre Modelos de OpenAI

### Análisis de Diferencias entre text-embedding-3-small y text-embedding-3-large

In [ ]:
# Visualizar embeddings del modelo large
print("=== Visualización: text-embedding-3-large ===")
df_pca_large, df_umap_large = create_visualization_comparison(embeddings_large, texts, "OpenAI Large")

# Comparar similitudes entre modelos
print("\n=== Análisis de Similitudes ===")
print("Calculando similitudes coseno entre los primeros 3 textos...\n")

for i in range(3):
    for j in range(i+1, 5):
        sim_small = calculate_cosine_similarity(embeddings_small[i], embeddings_small[j])
        sim_large = calculate_cosine_similarity(embeddings_large[i], embeddings_large[j])
        
        print(f"Textos {i+1} y {j+1}:")
        print(f"  Small: {sim_small:.4f}")
        print(f"  Large: {sim_large:.4f}")
        print(f"  Diferencia: {abs(sim_small - sim_large):.4f}")
        print(f"  Texto {i+1}: {texts[i][:50]}...")
        print(f"  Texto {j+1}: {texts[j][:50]}...")
        print()

## Paso 10: Análisis de Costos y Tokens

### Comparación Económica entre Modelos

Entender los costos es crucial para aplicaciones en producción:

In [ ]:
import tiktoken

def analyze_costs(texts):
    """
    Analiza costos de embedding para diferentes modelos
    """
    # Encoder para contar tokens
    encoding = tiktoken.encoding_for_model("gpt-4")
    
    # Contar tokens total
    total_tokens = sum(len(encoding.encode(text)) for text in texts)
    
    # Precios por 1K tokens (2025)
    prices = {
        "text-embedding-3-small": 0.00002,
        "text-embedding-3-large": 0.00013,
        "text-embedding-ada-002": 0.0001  # Modelo anterior para comparación
    }
    
    print(f"📊 Análisis de Costos para {len(texts)} textos")
    print(f"{'='*50}")
    print(f"Total de tokens: {total_tokens:,}")
    print(f"Promedio por texto: {total_tokens/len(texts):.1f} tokens")
    print()
    
    print("💰 Costo por Modelo:")
    for model, price in prices.items():
        cost = (total_tokens / 1000) * price
        print(f"  {model:25} ${cost:.6f}")
    
    print()
    print("📈 Escalamiento (para 1M textos similares):")
    estimated_tokens_1m = (total_tokens / len(texts)) * 1_000_000
    for model, price in prices.items():
        cost_1m = (estimated_tokens_1m / 1000) * price
        print(f"  {model:25} ${cost_1m:,.2f}")

# Analizar nuestros textos
analyze_costs(texts)

## Paso 11: Búsqueda Semántica

### Implementación de Similarity Search

La búsqueda semántica permite encontrar textos relacionados por significado, no solo por palabras clave:

**Proceso:**
1. Convertir consulta en embedding
2. Calcular similitud con todos los textos
3. Ordenar por relevancia
4. Mostrar resultados más similares

In [ ]:
def semantic_search(query, embeddings, texts, top_k=5, model="text-embedding-3-large"):
    """
    Realiza búsqueda semántica usando embeddings
    """
    # Generar embedding para la consulta
    embeddings_client = OpenAIEmbeddings(
        model=model,
        dimensions=3072  # Dimensiones reducidas para visualización
    )
    
    query_embedding = np.array(embeddings_client.embed_query(query))
    
    # Calcular similitudes
    similarities = []
    for i, doc_embedding in enumerate(embeddings):
        similarity = calculate_cosine_similarity(query_embedding, doc_embedding)
        similarities.append((i, similarity, texts[i]))
    
    # Ordenar por similitud
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return query_embedding, similarities[:top_k]

def visualize_search_results(query, query_embedding, similarities, embeddings, texts, model_name=""):
    """
    Visualiza los resultados de búsqueda semántica
    """
    # Preparar datos para visualización
    all_embeddings = np.vstack([embeddings, query_embedding.reshape(1, -1)])
    all_texts = texts + [f"🔍 CONSULTA: {query}"]
    
    # Crear categorías (incluyendo la consulta)
    categories = []
    for text in texts:
        if any(word in text.lower() for word in ['gato', 'perro', 'cachorro']):
            categories.append('Animales')
        elif any(word in text.lower() for word in ['computadora', 'smartphone', 'inteligencia artificial']):
            categories.append('Tecnología')
        elif any(word in text.lower() for word in ['cielo', 'montaña', 'océano']):
            categories.append('Naturaleza')
        else:
            categories.append('Educación')
    categories.append('Consulta')  # Para la consulta
    
    # PCA para visualización
    pca = PCA(n_components=2, random_state=42)
    embeddings_2d = pca.fit_transform(all_embeddings)
    
    # Crear DataFrame
    df = pd.DataFrame({
        'x': embeddings_2d[:, 0],
        'y': embeddings_2d[:, 1],
        'text': all_texts,
        'category': categories,
        'is_query': [False] * len(texts) + [True]
    })
    
    # Crear gráfico
    fig = px.scatter(
        df, x='x', y='y',
        color='category',
        symbol='is_query',
        size=[20 if is_query else 10 for is_query in df['is_query']],
        hover_data=['text'],
        title=f'Búsqueda Semántica {model_name}',
        labels={'x': f'PC1 ({pca.explained_variance_ratio_[0]:.1%})', 
                'y': f'PC2 ({pca.explained_variance_ratio_[1]:.1%})'}
    )
    
    # Destacar los resultados más similares
    top_indices = [s[0] for s in similarities[:3]]
    for idx in top_indices:
        fig.add_shape(
            type="circle",
            xref="x", yref="y",
            x0=embeddings_2d[idx][0] - 0.1, y0=embeddings_2d[idx][1] - 0.1,
            x1=embeddings_2d[idx][0] + 0.1, y1=embeddings_2d[idx][1] + 0.1,
            line=dict(color="red", width=2, dash="dash"),
            fillcolor="red", opacity=0.1
        )
    
    fig.show()
    
    return df

# Ejemplos de búsqueda semántica
queries = [
    "Me gusta estar en casa con mi mascota",
    "Dispositivos modernos y tecnología avanzada",
    "Paisajes naturales y belleza del mundo",
    "La semana pasada que hice con mi perro"
]

print("🔍 Ejemplos de Búsqueda Semántica")
print("="*50)

for query in queries:
    print(f"\n📝 Consulta: '{query}'")
    print("-" * 30)
    
    # Búsqueda con modelo large
    query_emb, results = semantic_search(query, embeddings_large, texts, top_k=3)
    
    print("🎯 Resultados más similares:")
    for i, (idx, similarity, text) in enumerate(results, 1):
        print(f"  {i}. Similitud: {similarity:.4f}")
        print(f"     Texto: {text}")
        print()
    
    # Visualizar solo la primera consulta
    if query == queries[0]:
        print("📊 Visualización de la primera búsqueda:")
        visualize_search_results(query, query_emb, results, embeddings_large, texts, "(text-embedding-3-large)")

In [ ]:
def semantic_search(query, embeddings, texts, top_k=5, model="text-embedding-3-large"):
    """
    Realiza búsqueda semántica usando embeddings
    """
    # Generar embedding para la consulta
    embeddings_client = OpenAIEmbeddings(
        model=model,
        dimensions=3072  # Dimensiones reducidas para visualización
    )
    
    query_embedding = np.array(embeddings_client.embed_query(query))
    
    # Calcular similitudes
    similarities = []
    for i, doc_embedding in enumerate(embeddings):
        similarity = calculate_cosine_similarity(query_embedding, doc_embedding)
        similarities.append((i, similarity, texts[i]))
    
    # Ordenar por similitud
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    return query_embedding, similarities[:top_k]


In [ ]:
# Generar embedding para la consulta
embeddings_client = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=3072  # Dimensiones reducidas para visualización
)

In [ ]:
query = "Me gusta estar en casa con mi mascota"

query_embedding = np.array(embeddings_client.embed_query(query))

query_embedding[:5]

In [ ]:
# Calcular similitudes
similarities = []
for i, doc_embedding in enumerate(embeddings_large):
    similarity = calculate_cosine_similarity(query_embedding, doc_embedding)
    similarities.append((i, similarity, texts[i]))

In [ ]:
similarities

In [ ]:
similarities.sort(key=lambda x: x[1], reverse=True)

In [ ]:
similarities[:5]

## 🎯 Resumen y Próximos Pasos

### ✅ Conceptos Aprendidos

1. **Embeddings**: Representaciones vectoriales que capturan significado semántico
2. **Modelos OpenAI**: Diferencias entre text-embedding-3-small y text-embedding-3-large
3. **Visualización**: PCA vs UMAP para entender agrupaciones semánticas
4. **Búsqueda Semántica**: Encontrar contenido por significado, no palabras exactas
5. **Costos**: Consideraciones económicas para proyectos reales

### 🚀 Aplicaciones en RAG

Los embeddings son fundamentales para:
- **Indexación**: Convertir documentos en vectores buscables
- **Retrieval**: Encontrar documentos relevantes para una pregunta
- **Re-ranking**: Mejorar la relevancia de resultados
- **Chunking Semántico**: Dividir textos por significado

### 🔬 Ejercicios Propuestos

1. **Experimenta con dimensiones**: Prueba diferentes valores (256, 1024, 1536)
2. **Textos bilingües**: Agrega textos en inglés y observa agrupaciones
3. **Dominios específicos**: Usa textos de tu área de expertise
4. **Threshold tuning**: Experimenta con diferentes umbrales de similitud

### 📚 Para Profundizar

- **LangChain Documentation**: Integración con bases de datos vectoriales
- **FAISS/Qdrant**: Escalamiento para millones de documentos
- **Evaluation**: Métricas para evaluar calidad de embeddings
- **Fine-tuning**: Personalización para dominios específicos